### 6.832 Project

Author: Gilhyun Ryou (ghryou@mit.edu), Seong Ho Yeon (syeon@mit.edu)


## Swing-up and balancing of a double inverted pendulum on a 2-D plannar quadrotor

### Case 1: Quadrotor pendulum

In [ ]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

import math
import numpy as np

from quadrotor_pendulum import QuadrotorPendulum, RunSimulation
from quadrotor_pendulum_visualizer import QuadrotorPendulumVisualizer
from quadrotor_pendulum_controller import OurController, LQRController, SampleController
from IPython.display import HTML

from quadrotor_pendulum_energy import analyze_energy
import matplotlib.pyplot as plt

# Make numpy printing prettier
np.set_printoptions(precision=3, suppress=True)

# Pendulum params. input_max = 2*hovering_thrust
mb = 1. # Default 1
lb = 0.2 # Default 0.2
m1 = 1. # Default 1
l1 = 0.2 # Default 0.2
g = 10.  # Default 10
input_max = 25. # Default 20
# Tree growth condition : 20, 18

quadrotor_plant = QuadrotorPendulum(
    mb = mb, lb = lb, m1 = m1, l1 = l1, 
    g = g, input_max = input_max)

x0 = np.array([0., 0., -math.pi, 0., 0., 0., 0., 0.])
xf = np.array([0., 0., 0, math.pi, 0., 0., 0., 0.])
uf = np.array([g, g])

our_ctrl = OurController(quadrotor_plant, xf=xf, uf=uf)
lqr_ctrl = LQRController(quadrotor_plant, xf=xf, uf=uf)
sample_ctrl = SampleController(quadrotor_plant)

# Run forward simulation from the specified initial condition
duration = 8.
input_log, state_log = \
    RunSimulation(quadrotor_plant,
              our_ctrl.feedback_rule,
              x0=x0,
              duration=duration,
              control_period = 0.005,
              simulation_period = 0.001)

V_set = []
Vd_set = []
E_set = []
Ep_set = []
for it in range(state_log.sample_times().shape[0]):
    x = state_log.data()[:, it]
    u = input_log.data()[:, it]
    x_t = x-lqr_ctrl.xf
#     x_t[2] = (x_t[2]+np.pi-1e-6)%(2*np.pi)-np.pi+1e-6
#     x_t[3] = (x_t[3]+np.pi-1e-6)%(2*np.pi)-np.pi+1e-6
    V = x_t.dot(lqr_ctrl.S).dot(x_t)
    u_t = np.array(u)
    Vd = 2.*x_t.dot(lqr_ctrl.S).dot(quadrotor_plant.evaluate_f(u_t,x,False))
    
    V_set.append(V)
    Vd_set.append(Vd)
    
    q = x[0:4]
    qd = x[4:8]
    E = 0.5*(quadrotor_plant.I1+quadrotor_plant.m1*quadrotor_plant.l1**2)*qd[3]**2-quadrotor_plant.m1*quadrotor_plant.g*quadrotor_plant.l1*math.cos(q[3])+quadrotor_plant.m1*quadrotor_plant.l1*qd[3]*(qd[0]*math.cos(q[3])+qd[1]*math.sin(q[3]))+(quadrotor_plant.mb+quadrotor_plant.m1)*quadrotor_plant.g*q[1]+0.5*quadrotor_plant.Ib*qd[2]**2+0.5*(quadrotor_plant.mb+quadrotor_plant.m1)*(qd[0]**2+qd[1]**2)
    Ep = 0.5*(quadrotor_plant.I1+quadrotor_plant.m1*quadrotor_plant.l1**2)*qd[3]**2-quadrotor_plant.m1*quadrotor_plant.g*quadrotor_plant.l1*math.cos(q[3])
    E_set.append(E)
    Ep_set.append(Ep)

init_idx = 0
t_stamp_i = 0
t_stamp_f = -1

plt.figure().set_size_inches(12, 10)
plt.subplot(4, 1, 1)
plt.plot(state_log.sample_times()[t_stamp_i:t_stamp_f], V_set[t_stamp_i:t_stamp_f])
plt.grid(True)
plt.ylabel("V")

plt.subplot(4, 1, 2)
plt.plot(state_log.sample_times()[t_stamp_i:t_stamp_f], Vd_set[t_stamp_i:t_stamp_f])
plt.grid(True)
plt.ylabel("Vd")

plt.subplot(4, 1, 3)
plt.plot(state_log.sample_times()[t_stamp_i:t_stamp_f], E_set[t_stamp_i:t_stamp_f])
plt.grid(True)
plt.ylabel("E")

plt.subplot(4, 1, 4)
plt.plot(state_log.sample_times()[t_stamp_i:t_stamp_f], Ep_set[t_stamp_i:t_stamp_f])
plt.grid(True)
plt.ylabel("Ep")

plt.figure().set_size_inches(8, 8)
plt.plot(state_log.data()[3, t_stamp_i:t_stamp_f], state_log.data()[7, t_stamp_i:t_stamp_f])
plt.grid(True)
plt.ylabel("theta1-theta1d")

# Visualize state and input traces
fig = plt.figure().set_size_inches(12, 10)
for i in range(8):
    plt.subplot(10, 1, i+1)
    plt.plot(state_log.sample_times()[t_stamp_i:t_stamp_f], state_log.data()[i, t_stamp_i:t_stamp_f])
    plt.grid(True)
    plt.ylabel("x[%d]" % i)
for i in range(2):
    plt.subplot(10, 1, i+9)
    plt.plot(input_log.sample_times()[t_stamp_i:t_stamp_f], input_log.data()[i, t_stamp_i:t_stamp_f])
    plt.grid(True)
    plt.ylabel("u[%d]" % i)
plt.xlabel("t")
plt.grid(True)

# print(state_log.data().shape)

# Visualize the simulation
analyze_energy(quadrotor_plant, state_log, input_log)

viz = QuadrotorPendulumVisualizer(quadrotor_plant)
ani = viz.animate(input_log, state_log, 60, repeat=True)
plt.close(viz.fig)
HTML(ani.to_html5_video())

# plt.show(ani.new_saved_frame_seq(), t = 0.1)

### Case 2: Quadrotor double pendulum

In [ ]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

import math
import numpy as np

from quadrotor_double_pendulum import QuadrotorDoublePendulum, RunSimulation
from quadrotor_double_pendulum_visualizer import QuadrotorDoublePendulumVisualizer
from quadrotor_double_pendulum_controller import OurController, LQRController, SampleController
from IPython.display import HTML
import matplotlib.pyplot as plt

# Make numpy printing prettier
np.set_printoptions(precision=3, suppress=True)

# Pendulum params. input_max = 2*hovering_thrust
mb = 1. # Default 1
lb = 0.2 # Default 0.2
m1 = 1. # Default 1
l1 = 0.2 # Default 0.2
m2 = 1. # Default 1
l2 = 0.2 # Default 0.2
g = 10.  # Default 10
input_max = 50. # Default 30

quadrotor_plant = QuadrotorDoublePendulum(
    mb = mb, lb = lb, m1 = m1, l1 = l1, 
    m2 = m2, l2 = l2, 
    g = g, input_max = input_max)


# Run forward simulation from the specified initial condition
duration = 5.

x0 = np.array([0., 0., -math.pi+0.1, 0., 0., 0., 0., 0., 0., 0.])
xf = np.array([0., 0., 0, math.pi, math.pi, 0., 0., 0., 0., 0.])

our_ctrl = OurController(quadrotor_plant, xf=xf)
lqr_ctrl = LQRController(quadrotor_plant, xf=xf)
sample_ctrl = SampleController(quadrotor_plant)

input_log, state_log = \
    RunSimulation(quadrotor_plant,
              our_ctrl.feedback_rule,
              x0=x0,
              duration=duration,
              control_period = 0.005,
              simulation_period = 0.001)

V_set = []
Vd_set = []
for it in range(state_log.sample_times().shape[0]):
    x = state_log.data()[:, it]
    u = input_log.data()[:, it]
    x_t = x-lqr_ctrl.xf
#     x_t[2] = (x_t[2]+np.pi-1e-6)%(2*np.pi)-np.pi+1e-6
#     x_t[3] = (x_t[3]+np.pi-1e-6)%(2*np.pi)-np.pi+1e-6
#     x_t[4] = (x_t[4]+np.pi-1e-6)%(2*np.pi)-np.pi+1e-6
    V = x_t.dot(lqr_ctrl.S).dot(x_t)
    u_t = np.array(u)
    Vd = 2.*x_t.dot(lqr_ctrl.S).dot(quadrotor_plant.evaluate_f(u_t,x,False))
    V_set.append(V)
    Vd_set.append(Vd)

init_idx = 0

plt.plot(state_log.sample_times()[init_idx:], V_set[init_idx:])
plt.grid(True)
plt.ylabel("V")

plt.plot(state_log.sample_times()[init_idx:], Vd_set[init_idx:])
plt.grid(True)
plt.ylabel("Vd")

# Visualize state and input traces
fig = plt.figure().set_size_inches(12, 12)
for i in range(10):
    plt.subplot(12, 1, i+1)
    plt.plot(state_log.sample_times(), state_log.data()[i, :])
    plt.grid(True)
    plt.ylabel("x[%d]" % i)
for i in range(2):
    plt.subplot(12, 1, i+11)
    plt.plot(input_log.sample_times(), input_log.data()[i, :])
    plt.grid(True)
    plt.ylabel("u[%d]" % i)
plt.xlabel("t")
plt.grid(True)

# Visualize the simulation
viz = QuadrotorDoublePendulumVisualizer(quadrotor_plant)
ani = viz.animate(input_log, state_log, 100, repeat=True)
plt.close(viz.fig)
HTML(ani.to_html5_video())
# plt.show(ani.new_saved_frame_seq())